<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Using TensorFlow Keras, Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn.

This is your "Hello World!" of TensorFlow.

### Example TensorFlow Starter Code

```python 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(3, activation='sigmoid', input_dim=2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

results = model.fit(X,y, epochs=100)

```

### Additional Written Tasks:
1. Investigate the various [loss functions](https://www.tensorflow.org/api_docs/python/tf/keras/losses). Which is best suited for the task at hand (predicting 1 / 0) and why? 
2. What is the difference between a loss function and a metric? Why might we need both in Keras? 
3. Investigate the various [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). Stochastic Gradient Descent (`sgd`) is not the learning algorithm dejour anyone. Why is that? What do newer optimizers such as `adam` have to offer? 

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import pandas as pd

data = { 'x1': [0,0,1,0,1,1,0],
         'x2': [0,1,0,1,0,1,0],
         'x3': [1,1,1,0,0,1,0],
         'y':  [0,1,1,1,1,0,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')
X = df[['x1', 'x2', 'x3']]
y = df[['y']]
y = y.replace(0, -1)

model = Sequential([
    Dense(3, activation='relu', input_dim=3),
    Dense(4, activation='relu'),
    Dense(1, activation='tanh')
])
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='squared_hinge', metrics=['acc'])
 
results = model.fit(X,y, epochs=100)

scores = model.evaluate(X, y)
print("\n")
print("Data Metrics:")
print(f"{model.metrics_names[0]}: {scores[0]}")
print(f"{model.metrics_names[1]}: {scores[1]*100}")

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1621 - acc: 0.1429
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1562 - acc: 0.1429
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 1.1452 - acc: 0.1429
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1302 - acc: 0.1429
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1120 - acc: 0.1429
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 1.0920 - acc: 0.1429
Epoch 7/100
1/1 [==============================] - 0s 1ms/step - loss: 1.0710 - acc: 0.1429
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 1.0504 - acc: 0.1429
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0336 - acc: 0.1429
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0169 - acc: 0.0000e+00
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0

In [70]:
##### Your Code Here #####
import numpy as np
results = pd.DataFrame(columns=['pred'], data=model.predict(X))
results.insert(loc=1, column='pred_category', value=np.round(model.predict(X)))
results.insert(loc=2, column='true', value=y.to_numpy())
results.replace(-1.0, 0).replace(-1, 0)

,pred,pred_category,true
0,-0.750799,0.0,0
1,0.888508,1.0,1
2,0.871719,1.0,1
3,0.914629,1.0,1
4,0.880087,1.0,1
5,-0.750799,0.0,0
6,-0.750799,0.0,0


### Build a Tensor Keras Perceptron

Try to match the architecture we used on Monday - inputs nodes and one output node. Apply this architecture to the XOR-ish dataset above. 

After fitting your model answer these questions: 

Are you able to achieve the same results as a bigger architecture from the first part of the assignment? Why is this disparity the case? What properties of the XOR dataset would cause this disparity? 

Now extrapolate this behavior on a much larger dataset in terms of features. What kind of architecture decisions could we make to avoid the problems the XOR dataset presents at scale? 

*Note:* The bias term is baked in by default in the Dense layer.

In [72]:
### Compare
model = Sequential([
    Dense(3, activation='relu', input_dim=3),
    Dense(1, activation='tanh')
])
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='squared_hinge', metrics=['acc'])
 
results = model.fit(X,y, epochs=100)

scores = model.evaluate(X, y)
print("\n")
print("Data Metrics:")
print(f"{model.metrics_names[0]}: {scores[0]}")
print(f"{model.metrics_names[1]}: {scores[1]*100}")
results = pd.DataFrame(columns=['pred'], data=model.predict(X))
results.insert(loc=1, column='pred_category', value=np.round(model.predict(X)))
results.insert(loc=2, column='true', value=y.to_numpy())
results.replace(-1.0, 0).replace(-1, 0)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1234 - acc: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1204 - acc: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1144 - acc: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 1.1058 - acc: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0953 - acc: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0835 - acc: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 842us/step - loss: 1.0707 - acc: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 913us/step - loss: 1.0575 - acc: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 943us/step - loss: 1.0442 - acc: 0.0000e+00
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0314 - acc: 0.0000e+00
Epoch 11/100
1/1 [===============

,pred,pred_category,true
0,-0.216152,-0.0,0
1,0.026244,0.0,1
2,0.786798,1.0,1
3,-0.217998,-0.0,1
4,0.927775,1.0,1
5,-0.217998,-0.0,0
6,-0.217998,-0.0,0


## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?